# Equation Solver And Tkinter GUI

In [1]:
from tensorflow.keras.models import model_from_json # type: ignore
import cv2
import numpy as np
import pandas as pd

print('Loading Model...')
model_json = open('model.json', 'r')
loaded_model_json = model_json.read()
model_json.close()
model = model_from_json(loaded_model_json)

print('Loading weights...')
model.load_weights("model.weights.h5")

Loading Model...
Loading weights...


In [2]:
labels = [ '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','+','-','x']

In [3]:
from PIL import Image,ImageDraw,ImageGrab

In [4]:
def activate_event(event):
        global lasx,lasy
        lasx,lasy=event.x,event.y   

In [5]:
def draw_smth(event):
    global lasx,lasy
    cv.create_line((lasx,lasy,event.x,event.y),fill='black',width=4)
    #cv.draw.line([x1, y1, x2, y2], fill="black", width=7)
    lasx,lasy=event.x,event.y

In [6]:
def save():
    filename="canvas.jpg"
    widget=cv
    x=root.winfo_rootx()+widget.winfo_x()+50
    y=root.winfo_rooty()+widget.winfo_y()+50
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_height()
    
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)

In [7]:
from sympy import *

class Solver:

    def __init__(self, equation):
        self.equation = str(equation)
        self.leftEqu = []

    def convertEquationIntoGeneralForm(self):

        leftSide, rightSide = '', ''
        equalIndx = self.equation.index('=')
        leftSide = self.equation[0:equalIndx]
        rightSide = self.equation[equalIndx+1:len(self.equation)]

        if rightSide[0].isalpha() or rightSide[0].isdigit():
            rightSide = '+' + rightSide

        for i in range(0, len(rightSide)):
            if rightSide[i] == '+':
                rightSide = rightSide[0:i] + '-' + rightSide[i+1:len(rightSide)]
            elif rightSide[i] == '-':
                rightSide = rightSide[0:i] + '+' + rightSide[i+1:len(rightSide)]
            leftSide += rightSide[i]

        self.equation = leftSide + '=' + '0'
        self.leftEqu = leftSide

    def solveEquation(self):

        self.convertEquationIntoGeneralForm()
        sympy_eq = sympify("Eq(" + self.equation.replace("=", ",") + ")")
        roots = solve(sympy_eq)
        
        return roots      

In [8]:
def predictFromArray(arr):
    result = model.predict_classes(arr)
    return result                    

In [9]:
def solution():
    img = cv2.imread('canvas.jpg',cv2.IMREAD_GRAYSCALE)
    img = ~img
    ret,thresh = cv2.threshold(img,127, 255,cv2.THRESH_BINARY)
    ctrs,_ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    img_data = []
    rects = []
    for c in cnt :
        x, y, w, h = cv2.boundingRect(c)
        rect = [x, y, w, h]
        rects.append(rect)
    final_rect = [i for i in rects]
    for r in final_rect:
        x,y,w,h = r[0],r[1],r[2],r[3]
        img = thresh[y:y+h+10, x:x+w+10]
        img = cv2.resize(img, (28, 28)) 
        img = np.reshape(img, (1, 28, 28)) 
        img_data.append(img)
        
    mainEquation=[]
    operation = ''
    for i in range(len(img_data)):
                img_data[i] = np.array(img_data[i])
                img_data[i] = img_data[i].reshape(-1, 28, 28, 1)
                result=predictFromArray(img_data[i])
                i=result[0]
                mainEquation.append(labels[i])
        
    StringEquation=""
    for i in range(len(mainEquation)):
            a=mainEquation[i]
            if(a.isdigit()==False and a.isalpha()==False and i<len(mainEquation)-1):
                if(a==mainEquation[i+1]=='-'):
                    StringEquation+='='
                else:
                    StringEquation+=a
            if(a.isalpha()==True):
                if(i>0):
                    if(mainEquation[i-1].isdigit()):
                        StringEquation+="*"+a
                    else:
                        StringEquation+=a
                else:
                    StringEquation+=a
            if(a.isdigit()==True):
                if(i>0):
                    if(mainEquation[i-1].isdigit()):
                        StringEquation+=a
                    elif(mainEquation[i-1].isalpha()):
                         StringEquation+="^"+a
                    else:
                        StringEquation+=a
                else:
                    StringEquation+=a
            
    newStr=""
    l=list(StringEquation)   
    for i in range(len(l)):
            if(l[i]=="="):
                newStr=l[:i+1]+l[i+2:]
    print(newStr)
    equ=""
    for i in newStr:
            equ+=i
    solution=Solver(equ)
            
    str1=''        
    roots=solution.solveEquation()
    st=[]
    for i in roots:
        i=str(i)
        st.append(i)
       
    str1=', '.join(st)
 
    solving(equ,str1)      

In [10]:
import numpy as np
from tkinter import *
import cv2
from PIL import Image, ImageGrab
from tkinter import messagebox

# Define a dummy function for predictFromArray to simulate predictions
# Replace this with your trained model's prediction function
def predictFromArray(img_data):
    # Dummy implementation: Returns random digits/operators
    # Replace with model.predict(img_data)
    return [np.random.randint(0, 10)]  # Simulate a digit prediction

# Dummy labels: Map model output indices to characters (extend for operators as needed)
labels = [str(i) for i in range(10)] + ['+', '-', '*', '/']

# Solver class to simulate solving the equation
class Solver:
    def __init__(self, equation):
        self.equation = equation

    def solveEquation(self):
        try:
            return [eval(self.equation)]  # Evaluate the equation
        except:
            return ["Invalid equation"]

# Tkinter setup
root = Tk()
root.resizable(0, 0)
root.title('Equation Solver')

# Global variables for drawing
lasx, lasy = None, None

# Canvas for drawing
cv = Canvas(root, width=1200, height=500, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)
cve2 = Label(root)
cve = Label(root, font=("Helvetica", 16))
cve.grid(row=0, column=1, pady=1, padx=1)
cve2.grid(row=1, column=1, pady=1, padx=1)

# Activate drawing
def activate_event(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y

# Draw on canvas
def draw_smth(event):
    global lasx, lasy
    x, y = event.x, event.y
    cv.create_line((lasx, lasy, x, y), fill='black', width=5)
    lasx, lasy = x, y

# Directly process and predict the equation from the canvas
def solution():
    try:
        # Capture the canvas content as an image
        x = root.winfo_rootx() + cv.winfo_x()
        y = root.winfo_rooty() + cv.winfo_y()
        x1 = x + cv.winfo_width()
        y1 = y + cv.winfo_height()
        img = ImageGrab.grab().crop((x, y, x1, y1))
        
        # Convert the image to grayscale for processing
        img = img.convert("L")  # Convert to grayscale
        img = np.array(img)  # Convert to NumPy array
        img = ~img  # Invert colors
        _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        ctrs, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

        img_data = []
        rects = []

        # Extract bounding boxes
        for c in cnt:
            x, y, w, h = cv2.boundingRect(c)
            rects.append([x, y, w, h])

        # Process each character
        for r in rects:
            x, y, w, h = r[0], r[1], r[2], r[3]
            char_img = thresh[y:y + h + 10, x:x + w + 10]
            char_img = cv2.resize(char_img, (28, 28))
            char_img = np.reshape(char_img, (1, 28, 28))
            img_data.append(char_img)

        # Predict and build the equation
        mainEquation = []
        for data in img_data:
            data = np.expand_dims(data, axis=-1)
            data = np.expand_dims(data, axis=0)
            result = predictFromArray(data)
            mainEquation.append(labels[result[0]])

        # Build the equation string
        equation = "".join(mainEquation)
        solution = Solver(equation)
        roots = solution.solveEquation()

        # Display results
        solving(equation, ", ".join(map(str, roots)))

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Display the results
def solving(equ, roots):
    cve2.configure(text='Your Equation is: ' + equ)
    cve.configure(text='Result: ' + roots)

# Bind drawing events
cv.bind('<Button-1>', activate_event)
cv.bind('<B1-Motion>', draw_smth)

# Buttons
btn_predict = Button(text="Predict", command=solution, bg='#6495ED', fg='White')
btn_predict.grid(row=2, column=1, pady=1, padx=1)


cv.bind('<Button-2>',activate_event)
cv.bind('<B2-Motion>',draw_smth)
btn_save=Button(text="Save",command=save,bg='#6495ED',fg='White')
btn_save.grid(row=2,column=0,pady=1,padx=1)

# Run the application
root.mainloop()
